In [ ]:
import numpy as np
import cv2
import statistics
from scipy.fftpack import fft, dct, idct
from scipy.signal import savgol_filter
import sys
import json
import math
import matplotlib.pyplot as plt
import collections
import pickle
import glob
import os
from scipy import spatial
import shutil

names = ['Ahmad','Alireza','Carlos','Rahul','Yin']

folder_names = ["Pizza","Turkey"]

for FOLDER_NAME in folder_names:

    for NAME in names:
        
        NAME = NAME+"_"+FOLDER_NAME

        frame_list = []
        cap = cv2.VideoCapture(FOLDER_NAME+"/"+NAME+".avi")
        while True:
            ret,frame = cap.read()
            if ret == True:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame_list.append(gray)

            else:
                break
        cap.release()

        # implement 2D DCT
        def dct2(a):
            return dct(dct(a.T, norm='ortho').T, norm='ortho')

        # implement 2D IDCT
        def idct2(a):
            return idct(idct(a.T, norm='ortho').T, norm='ortho') 

        def zigzag(a):
            rows = a.shape[0]
            columns = a.shape[1]
            solution=[[] for i in range(rows+columns-1)]
            matrix = a
            r_mat = []  
            for i in range(rows):
                for j in range(columns):
                    summer=i+j
                    if(summer%2 ==0):


                        solution[summer].insert(0,matrix[i][j])
                    else:


                        solution[summer].append(matrix[i][j])


            # print the solution as it as
            for i in solution:
                for j in i:
                    r_mat.append(j)

            return r_mat

        mean_list = []
        stdev_list = []
        blurred_frames = []
        for i in range(0,len(frame_list),15):
            img = frame_list[i]
            img = cv2.GaussianBlur(img, (205, 205), 0)
            blurred_frames.append(img)
            dct_transform = dct2(img)
            z = zigzag(dct_transform)
            z = z[:500]
        #     m = statistics.mean(z)
            m = z
            std = statistics.stdev(z)
            mean_list.append(m)
            stdev_list.append(std)
        #     print("Frame number {} Mean is {} Stdev is {}".format(i,m,std))

        # smooth_mean = savgol_filter(mean_list, 51, 3) 

        total_dict = {}
        scenes_dict = {
            'scene_mean_list' : [],
            'scene_frame_numbers' : [] 
        }

        temp_list = []
        frame_numer_list = []
        scene_counter = 0
        for i in range(len(mean_list)-1):

        #     if(abs(mean_list[i+1]-mean_list[i])<4):
        #         temp_list.append(mean_list[i])
        #         frame_numer_list.append(i*5)

            m1 = mean_list[i+1]
            m2 = mean_list[i]

            d =  spatial.distance.cosine(m1, m2)

            print(d)

            if(d<0.05):
                temp_list.append(mean_list[i])
                frame_numer_list.append(i*15)
            else:
                if(len(temp_list)>2):
                    scenes_dict['scene_mean_list'] = temp_list
                    scenes_dict['scene_frame_numbers'] = frame_numer_list
                    total_dict[scene_counter] = scenes_dict
                    scene_counter+=1
                scenes_dict = {}
                temp_list = []
                frame_numer_list = []

        with open('TotalDict.json','w') as f:
            json.dump(total_dict,f)

        foundStart = False
        totalList = []
        with open(FOLDER_NAME+"/"+NAME+'.txt','r') as f:
            for line in f:
                if(line[0] == 'T'):
                    foundStart = True
                else:
                    if(foundStart==True):
                        tempDict = {}

                        l = line.split('\t')

                        tempDict['ts'] = int(l[0])
                        x = round(float(l[3])/1280,2)
                        y = round(float(l[4])/960,2)
                        tempDict['gp'] = (x,y)
                        tempDict['imageNum'] = int(l[5])
                        if(x>=0 and y>=0 and x<=1 and y<=1):
                            totalList.append(tempDict)

        temp_list = []
        for i in range(len(totalList)):
            tempDict = totalList[i]
            if(int(tempDict['imageNum'])<=len(frame_list)):
                temp_list.append(tempDict)

        userDataForGP = {}
        imageNumPrev = 0
        temp_gp_data = []

        for i in range(len(temp_list)):

            tempDict = temp_list[i]
            imageNum = int(tempDict['imageNum'])

            if(imageNum!= imageNumPrev or imageNum==0):
                userDataForGP[imageNumPrev] = temp_gp_data
                imageNumPrev = imageNum
                temp_gp_data = []

            gp_data = list(tempDict['gp'])
            temp_gp_data.append(gp_data)



            imageNumPrev = imageNum



        userDataForTS = {}
        imageNumPrev = 0
        temp_ts_data = []

        for i in range(len(temp_list)):

            tempDict = temp_list[i]
            imageNum = int(tempDict['imageNum'])

            if(imageNum!= imageNumPrev or imageNum==0):
                userDataForTS[imageNumPrev] = temp_ts_data
                imageNumPrev = imageNum
                temp_ts_data = []

            ts_data = int(tempDict['ts'])
            temp_ts_data.append(ts_data)



            imageNumPrev = imageNum

        import json

        with open("DataForGP.json","w") as f:
            json.dump(userDataForGP,f)

        with open("DataForTS.json","w") as f:
            json.dump(userDataForTS,f)

        temp_list = np.array(temp_list)

        with open("AllData.npy",'wb') as f:
            np.save(f,temp_list)

        a = [[2,2,2],[2,2,2]]

        a = np.array(a)
        a.shape

        temp_frame = frame_list[0]
        temp_frame = np.array(temp_frame)
        temp_frame.shape

        w,h = temp_frame.shape

        allGPList = [] 

        for key in userDataForGP.keys():
            l = userDataForGP[key]

            for i in l:
                allGPList.append(i)

        allGPList = np.array(allGPList)

        midGP = np.mean(allGPList,axis=0)

        finalDispersion = {}

        for key in userDataForGP.keys():
            l = userDataForGP[key]
            temp = []
            for i in l:
                x_change = i[0]-midGP[0]
                y_change = i[1]-midGP[1]

                valM = (x_change**2)+(y_change**2)

                magnitudeOfChange = math.sqrt(valM)

                temp.append(magnitudeOfChange)

            finalDispersion[key] = temp


        with open("DispersionMetric.json","w") as f:
            json.dump(finalDispersion,f)

        with open('TotalDict.json','r') as f:
            sceneInfo = json.load(f)


        frames = frame_list

        with open("DispersionMetric.json",'rb') as f:
            dispersionDict = json.load(f)

        dispDictKeyList = dispersionDict.keys()

        print(sceneInfo)

        timeSpans = {}


        dispersionExceptScenes = {}

        start_index = 0
        for key in sceneInfo.keys():
            scene = sceneInfo[key]
            f = scene['scene_frame_numbers']
            start = start_index+1
            end = f[0]

            dispersionList = []
            for i in range(start,end,1):

                if(str(i) in dispDictKeyList):

                    disp = dispersionDict[str(i)]
                    dispersionList.extend(disp)

            dispersionExceptScenes[key] = dispersionList
            timeS = (end-start)/24
            timeSpans[key] = timeS


            print(start_index, end)
            start_index = f[-1]

        betweenScenesData = {}    

        for key in dispersionExceptScenes.keys():

            tempDict = {}
            t = timeSpans[key]

            dispersions = dispersionExceptScenes[key]


            dispMid = abs(round(np.mean(dispersions),2))

            dispCounter = 0

            for i in dispersions:
                if(i>dispMid):
                    dispCounter+=1

            tempDict['DispersionCount'] = dispCounter
            tempDict['TimeGap'] = t


            betweenScenesData[key] = tempDict


        with open("DataBetweenScenes.json","w") as f:
            json.dump(betweenScenesData,f)

        betweenScenesData

        dataBwScenes = betweenScenesData

        disp_count = []
        for key in dataBwScenes.keys():
            tempD = dataBwScenes[key]
            disp_count.append(int(tempD['DispersionCount']))

        disp_count = np.array(disp_count)
        mean_disp = np.mean(disp_count)
        print(mean_disp)


        refined_dict = {}
        for key in betweenScenesData.keys():
            if(int(key)>0):
                data = dataBwScenes[key]
                refined_dict[int(key)-1] = data

        with open("DataBetweenScenes.json","w") as f:
            json.dump(refined_dict,f)

        frames = frame_list

        with open('TotalDict.json','r') as f:
            sceneInfo = json.load(f)


        orb = cv2.xfeatures2d.SURF_create()


        des_prev = []
        bf = cv2.BFMatcher()
        key_prev = None
        kp_prev = []
        counter = 0
        sameSURFScenes = {}
        listOfSameScenes = []
        for key in sceneInfo.keys():

            sceneF = sceneInfo[key]['scene_frame_numbers']
            mid = len(sceneF)//2
            mid = sceneF[mid]
            f = frames[mid]
            kp_next, des_next = orb.detectAndCompute(f, None)
            key_next = key

            if(len(des_prev)>0):
                matches = bf.knnMatch(des_prev,des_next, k=2)
                good = []
                for m,n in matches:
                    if m.distance < 0.65*n.distance:
                        good.append([m])
                        a=len(good)
                        percent=(a*100)/len(kp_prev)
                        if percent>=100.00:
                            break

                print(percent)
                if(percent>1):
                    listOfSameScenes.append(key_prev)
                    listOfSameScenes.append(key_next)
                else:
                    if(len(listOfSameScenes)>0):
                        sameSURFScenes[counter] = set(listOfSameScenes)
                        listOfSameScenes = []
                        counter+=1

            des_prev = des_next
            key_prev = key_next
            kp_prev = kp_next


        if(len(listOfSameScenes)>0):
            sameSURFScenes[counter] = set(listOfSameScenes)


        all_done_scenes = []
        finalScenes = {}
        for key in sameSURFScenes.keys():
            sceneAll = list(sameSURFScenes[key])
            for i in range(len(sceneAll)):
                sceneAll[i] = int(sceneAll[i])

            sceneAll.sort()
            start_scene = str(sceneAll[0])
            end_scene = str(sceneAll[-1])

            print(sceneAll)

            all_done_scenes.extend(sceneAll)

            start = sceneInfo[start_scene]['scene_frame_numbers'][0]
            end = sceneInfo[end_scene]['scene_frame_numbers'][-1]
            frames_list = [start,end]

            finalScenes[int(start_scene)] = frames_list

            print(start,end)

        for key in sceneInfo.keys():
            if int(key) not in all_done_scenes:
                scene_frames = sceneInfo[key]['scene_frame_numbers']
                finalScenes[int(key)] = scene_frames


        finalScenes = collections.OrderedDict(sorted(finalScenes.items()))

        with open("scenesAfterSURF.json","w") as f:
            json.dump(finalScenes,f)

        gap_dict = {}
        k = list(finalScenes.keys())

        for i in range(len(k)-1):
          start = k[i]
          end = k[i+1]

          #must be from start upto end-1
          temp_d = {}
          dc = []
          tg = 0
          for i in range(start,end,1):
            d = refined_dict[i]
            dc.append(d['DispersionCount'])
            tg+= d['TimeGap']

          md = max(dc)

          print(md,tg)

          temp_d['DispersionCount'] = md
          temp_d['TimeGap'] = tg

          gap_dict[start] = temp_d





        frames = frame_list

        with open("scenesAfterSURF.json","r") as f:
            scenes = json.load(f)

        keys = list(scenes.keys())

        groupedScenes = {}
        counter = 0
        temp_group = []

        for i in range(len(keys)-1):

          s = keys[i]
          e = keys[i+1] 

          g = gap_dict[int(s)]

          if(g['TimeGap']>5):
            if(g['DispersionCount']<50):
              temp_group.append(s)
              temp_group.append(e)

            else:
              #break scenes
              temp_group.append(s)
              groupedScenes[s] = list(set(temp_group))
              temp_group = []


          else:
            temp_group.append(s)
            temp_group.append(e)


        if(len(temp_group)>1):
          groupedScenes[s] = list(set(temp_group))


        groupedScenes = collections.OrderedDict(sorted(groupedScenes.items()))

        finalScenes = {}
        for i in groupedScenes.keys():
          key = i
          l = groupedScenes[key]
          l.sort()

          if(len(l)>1):
            start = scenes[l[0]][0]
            end = scenes[l[-1]][-1]

            finalScenes[key] = [start,end]  

          else:
            scene_frames = scenes[l[0]]
            start = scene_frames[0]
            end = scene_frames[-1]
            finalScenes[key] = [start,end]


        with open("FinalScenes.json","w") as f:
            json.dump(finalScenes,f,indent=4)

        frames = frame_list

        with open("TotalDict.json","r") as f:
            totalDict = json.load(f)

        with open("FinalScenes.json","r") as f:
            finalScenes = json.load(f)

        with open("DataForGP.json","r") as f:
            dataForGP = json.load(f)

        mapper = {}
        sceneKeys = totalDict.keys()
        for key in finalScenes.keys():
            f = finalScenes[key]
            start_frame = f[0]
            end_frame = f[-1]


            l = []

            for k in sceneKeys:

                temp = totalDict[k]['scene_frame_numbers']

                if(int(temp[0])>=int(start_frame) and int(temp[-1])<=int(end_frame)):
                    l.append(k)

                else:
                    mapper[key] = l   

        sceneBreakdown = {}
        for key in mapper.keys():
            l = mapper[key]
            tempDict = {}


            if(len(l)>1):
                for i in range(len(l)-1):
                    sceneFrames1 = totalDict[l[i]]['scene_frame_numbers']
                    sceneFrames2 = totalDict[l[i+1]]['scene_frame_numbers']
                    transitionDuration = sceneFrames2[0]-sceneFrames1[-1]
                    print(transitionDuration)

                    transition_start = sceneFrames1[-1]
                    transition_end = sceneFrames2[0]

                    transition = [transition_start,transition_end]

                    tempDict['scene'+str(i)] = [sceneFrames1[0],sceneFrames1[-1]]
                    tempDict['transition'+str(i)] = transition

                tempDict['scene'+str(i+1)] = [sceneFrames2[0],sceneFrames2[-1]]

                sceneBreakdown[key] = tempDict

            else:
                try:
                    sceneFrames1 = totalDict[l[0]]['scene_frame_numbers']
                    tempDict['scene'] = [sceneFrames1[0],sceneFrames1[-1]]
                    sceneBreakdown[key] = tempDict
                except:
                    pass



        with open("SceneBreakdown.json","w") as f:
            json.dump(sceneBreakdown,f)

        with open("SceneBreakdown.json","r") as f:
            sceneBreakdown = json.load(f)



        user = NAME
        part = "1"
        scene_breakdown_location = "SceneBreakdown.json"
        data_for_gp_location = "DataForGP.json"

        frames = frame_list

        with open(scene_breakdown_location,"r") as f:
            sceneBreakdown = json.load(f)

        with open(data_for_gp_location,"r") as f:
            dataForGP = json.load(f)

        def __zoom(img, scale, center=None,mid = None):
            # actual function to zoom
            height, width = img.shape[:2]
            if center is None:
                #   Calculation when the center value is the initial value
                center_x = int(width / 2)
                center_y = int(height / 2)

                # center_x = 0.5
                # center_y = 0.5
                radius_x, radius_y = int(width / 2), int(height / 2)
            else:
                #   Calculation at a specific location
                rate = 0.5
                try:
                  center_x, center_y = center
                  center_x = center_x*width
                  center_y = center_y*height
                except:
                  center_x = int(width / 2)
                  center_y = int(height / 2)

            print("CENTER",center_x,center_y)


            #   Calculate centroids for ratio range
            if center_x < width * (1-rate):
                center_x = width * (1-rate)
            elif center_x > width * rate:
                center_x = width * rate
            if center_y < height * (1-rate):
                center_y = height * (1-rate)
            elif center_y > height * rate:
                center_y = height * rate

            center_x, center_y = int(center_x), int(center_y)
            left_x, right_x = center_x, int(width - center_x)
            up_y, down_y = int(height - center_y), center_y
            radius_x = min(left_x, right_x)
            radius_y = min(up_y, down_y)

            # Actual zoom code
            radius_x, radius_y = int(scale * radius_x), int(scale * radius_y)

            # size calculation
            min_x, max_x = center_x - radius_x, center_x + radius_x
            min_y, max_y = center_y - radius_y, center_y + radius_y

            # Crop image to size
            cropped = img[min_y:max_y, min_x:max_x]
            # Return to original size
            new_cropped = cv2.resize(cropped, (width, height))

        #     new_cropped = cv2.circle(new_cropped, (int(mid[0]*width),int(mid[1]*height)), radius=0, color=(0, 0, 0), thickness=2)
            new_cropped = cv2.drawMarker(new_cropped, (int(mid[0]*width),int(mid[1]*height)), markerType=cv2.MARKER_CROSS,markerSize=30, 
                                          color=(0, 0, 0),thickness=3, line_type=cv2.LINE_AA)

            return new_cropped

        def progressiveZoomIn(frame_start,frame_end,center,initial,dataForGP):
            list_of_modified_frames = []
            initial = initial
            update_factor = 0.005
            if(frame_end-frame_start)<20:

              #When such a small scene, no zoom

                update_factor = 0
                #0.05

            # print("CENTER",center)

            for i in range(frame_start,frame_end,1):
                f = frames[i]
                try:
                  gp_value = dataForGP[str(i)]
                except:
                  gp_value = [[0.5,0.5],[0.5,0.5]]
                gp_value = np.array(gp_value)

                if(gp_value.shape[0]==2):
                    mid = np.mean(gp_value,axis=0)

                else:
                    mid = np.array([0.5,0.5])

                zoomed = __zoom(f,initial,center,mid)
                list_of_modified_frames.append(zoomed)
                # list_of_modified_frames.append(f)

                if(initial>0.75):
                    initial -= update_factor

            return list_of_modified_frames

        def progressiveZoomOut(frame_start,frame_end,center,initial,dataForGP):
            list_of_modified_frames = []
            initial = initial
            update_factor = 0.01
            if(frame_end-frame_start)<20:
                update_factor = 0.05
                #0.05

            # print("CENTER", center)

            for i in range(frame_start,frame_end,1):
                f = frames[i]



                try:
                  gp_value = dataForGP[str(i)]
                except:
                  gp_value = [[0.5,0.5],[0.5,0.5]]

                gp_value = np.array(gp_value)

                if(gp_value.shape[0]==2):
                    mid = np.mean(gp_value,axis=0)

                else:
                    mid = np.array([0.5,0.5])

                zoomed = __zoom(f,initial,center,mid)
                list_of_modified_frames.append(zoomed)
                # list_of_modified_frames.append(f)
                if(initial<1):
                    initial += update_factor

            return list_of_modified_frames

        finalZoomedAndPanned = {}

        for key in sceneBreakdown.keys():
            tempHolder = sceneBreakdown[key]

            entirePannedScene = []

            for k in tempHolder.keys():
                # print(k)
                if(k[:5]=="scene"):
                    #Actually a scene
                    start_frame = tempHolder[k][0]
                    end_frame = tempHolder[k][-1]
                    gp_data = []

                    for i in range(start_frame,end_frame,1):
                        try:
                          gp_data.extend(dataForGP[str(i)])
                        except:
                          pass

                    # print(gp_data)

                    #take central ROI of data gp
                    gp_data = np.array(gp_data)  
                    mean = np.mean(gp_data,axis=0)

                    # print(gp_data)

                    print("MEAN",mean)

                    listOfModified = progressiveZoomIn(start_frame,end_frame,mean,1,dataForGP)
                    entirePannedScene.extend(listOfModified)

                else:
                    #Transition Frame
                    start_frame = tempHolder[k][0]
                    end_frame = tempHolder[k][-1]
                    gp_data = []

                    for i in range(start_frame,end_frame,1):
                      try:
                        gp_value = dataForGP[str(i)]
                      except:
                        gp_value = [[0.5,0.5],[0.5,0.5]]

                    #take central ROI of data gp
                    gp_data = np.array(gp_data)  
                    mean = np.mean(gp_data,axis=0)
        #             print(mean)

                    listOfModified = progressiveZoomOut(start_frame,end_frame,mean,0.75,dataForGP)
                    entirePannedScene.extend(listOfModified)

            finalZoomedAndPanned[key] = entirePannedScene



        h = 960
        w = 1280
        for key in finalZoomedAndPanned.keys():
            try:
                os.makedirs("User"+user+"/Part"+part+"/"+key+"/")
            except:
                print("Error making dir")
                print("User"+user+"/Part"+part+"/"+key+"/")
                pass
            frames = finalZoomedAndPanned[key]
            print(len(frames))

            video_name = "User"+user+"/Part"+part+"/"+key+"RT_FINAL.mp4"
            video = cv2.VideoWriter(video_name, 0x7634706d, 24, (1280, 960))

            for x in range(len(frames)):
                filename = "User"+user+"/Part"+part+"/"+key+"/"+str(x)+"NR.jpg"
                cv2.imwrite(filename,frames[x])

                temp_f = "User"+user+"/Part"+part+"/"+key+"/"+str(x)+"NR.jpg"
                frame = cv2.imread(temp_f)
                video.write(frame)
            video.release()
            
            shutil.rmtree("User"+user+"/Part"+part+"/"+key)

    del frame_list